<a href="https://colab.research.google.com/github/xuede/AuthorGPT-4.5-Turbo/blob/main/AuthorGPT_4_5_Turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AuthorGPT-4.5-Turbo

Github repo: https://github.com/xuede/AuthorGPT-4_5-Turbo

This was only made possible by earlier work by [mshumer/gpt-author](https://github.com/mshumer/gpt-author).

I don't have an Anthropic API key, but with GPT-4.5-Turbo's 128k token context window, I wanted to generate the entire book with GPT-4.5-Turbo.


Generate an entire novel in minutes, and automatically package it as an e-book.

To generate a book:
*   ***Set the secrets for your STABILITY_API_KEY and OPENAI_API_KEY in Colab*** by clicking the key icon on the left. The names are as they appear above in caps. The value is the actually API key.
*   Make sure the toggle switch is to the right and blue (not grayed out).



1.  If you want your ebook saved to Google Drive, run the first cell. If you don't want to save it to the cloud, don't run it and you will be provided a download link at the end.
2.  Fill in the prompt, number of chapters, and writing style in the last cell.
3.  Run the rest of the cells! When complete you'll have a download link and if you chose to, a copy in your Google Drive at /AI/AuthorGPT/{title}_YYYY_MM_DD.




#Optionally save to Google Drive

If you want to save your story to your google drive click here and then run all cells. It will save the file to in /content/drive/mydrive/AI/AuthorGPT/{title}_YYMMDD

In [1]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
#@title Enter Novel Details { run: "auto", vertical-output: true }
num_chapters = 10 #@param {type:"integer"}
user_input_prompt = "Imagine a children's book where all the characters are boogers." #@param {type:"string"}
writing_style = 'Written for a 4th grade reading level, make it humorous, tongue-in-cheek, accessible and page-turning/captivating. It is both silly and smart.' #@param {type:"string"}

static_text = """
 Welcome to a new session. In this instance, your attention is needed on the mindful use of specific words and phrases that have been identified as frequently used in previous interactions. These include 'delve', 'tapestry', 'weave', 'chronicle', 'veil', 'shadows', 'whisper', 'luminous', 'ethereal', 'verdant', 'resplendent', 'meander', 'soar', 'plunge', 'cascade', 'twilight', 'horizon', 'echo', 'silhouette', 'poignant', 'melancholic', 'exuberant', 'serene', 'existential', 'transcendent', 'ephemeral', 'enigma', 'luminescent', 'iridescent' along with 'silent', 'unseen', 'light'. While these words are not banned, they should be used with discernment and only when they significantly enhance the communication. Guidelines for language use are as follows: Keyword Awareness: Actively recognize these specified words and phrases at the start of each interaction, considering them as terms to be used sparingly. Alternative Vocabulary: Develop and use a list of alternative words and phrases that can effectively convey similar meanings or functions as the flagged words. This alternative list should be a go-to resource in your language composition. Contextual Judgment: Carefully evaluate the necessity of using these words in each context. Opt for alternatives unless these words are crucial for conveying a specific and unique meaning. Diverse Language Styles: Strive to employ a broad array of linguistic styles and tones. This strategy helps in avoiding the overuse of any particular group of words or phrases, ensuring a rich and varied language texture. Reflection and Adaptation: Post-composition, review your responses to check the frequency and appropriateness of the flagged words. Adjust the content to achieve a balanced and diverse language usage. Continual Learning: Each new conversation is an opportunity to refine your language use based on feedback and the specific context, despite the stateless nature of the interactions. This approach is designed to foster a more varied and nuanced use of language, ensuring that your communication remains effective and engaging while avoiding over-reliance on certain words and phrases.
"""
# Combine user input with the static text
prompt = user_input_prompt + "\n\n" + static_text

# Now, 'prompt' contains both the user input and the static text


## Install Dependancies and Functions Setup


Install Dependancies

In [ ]:
# @title
!pip install cohere
!pip install tiktoken
!pip install openai
!pip install EbookLib

Imports and set OpenAI and Stability keys from Secrets

In [25]:
# @title
import openai
import os
from ebooklib import epub
import base64
import os
import requests
from google.colab import userdata
import openai

# Retrieve the OpenAI API key
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set the API key for OpenAI
openai.api_key = OPENAI_API_KEY

# Similarly, if you need to retrieve another key, for example, for Stability AI
STABILITY_API_KEY = userdata.get('STABILITY_API_KEY')

#openai.api_key = "OPENAI_API_KEY" # get it at https://platform.openai.com/
#stability_api_key = "STABILITY_API_KEY" # get it at https://beta.dreamstudio.ai/

Novel Writing Functions Setup: AI-Assisted Plot Generation and Chapter Development

In [26]:
# @title
import openai
import random
import json
import ast

def print_step_costs(response, model):
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens

    # Set default values
    input_per_token = 0
    output_per_token = 0

    if model == "gpt-4":
        input_per_token = 0.03 / 1000
        output_per_token = 0.06 / 1000
    elif model == "gpt-4-1106-preview":
        input_per_token = 0.01 / 1000
        output_per_token = 0.03 / 1000

    input_cost = input_tokens * input_per_token
    output_cost = output_tokens * output_per_token

    total_cost = input_cost + output_cost
    print('Step Cost (OpenAI):', total_cost)


def generate_plots(prompt):
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a creative assistant that generates engaging surreal novel plots."},
            {"role": "user", "content": f"Generate 10 surreal novel plots based on this prompt: {prompt}"}
        ]
    )

    print_step_costs(response, "gpt-4-1106-preview")

    return response.choices[0].message.content.split('\n')

def select_most_engaging(plots):
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are an expert in writing fantastic surrealist novel plots."},
            {"role": "user", "content": f"Here are a number of possible plots for a new novel: {plots}\n\n--\n\nNow, write the final plot that we will go with. It can be one of these, a mix of the best elements of multiple, or something completely new and better. The most important thing is the plot should be fantastic, unique, and engaging."}
        ]
    )

    print_step_costs(response, "gpt-4-1106-preview")

    return response.choices[0].message.content


def improve_plot(plot):
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are an expert in improving and refining story plots."},
            {"role": "user", "content": f"Improve this plot: {plot}"}
        ]
    )

    print_step_costs(response, "gpt-4-1106-preview")

    return response.choices[0].message.content



def get_title(improved_plot):
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are an expert writer."},
            {"role": "user", "content": f"Here is the plot: {improved_plot}\n\nWhat is the title of this book? Just respond with the title, do nothing else."}
        ]
    )

    print_step_costs(response, "gpt-4-1106-preview")

    return response.choices[0].message.content


def write_first_chapter(plot, first_chapter_title, writing_style):

    response = openai.chat.completions.create(
          model="gpt-4-1106-preview",
          messages=[
              {"role": "system", "content": "You are a world-class surrealist writer."},
              {"role": "user", "content": f"Here is the high-level plot to follow: {plot}\n\nWrite the first chapter of this novel: `{first_chapter_title}`.\n\nMake it incredibly unique, engaging, and well-written.\n\nHere is a description of the writing style you should use: `{writing_style}`\n\nInclude only the chapter text. There is no need to rewrite the chapter name."}
          ]
      )

    print_step_costs(response, "gpt-4-1106-preview")

    improved_response = openai.chat.completions.create(
          model="gpt-4-1106-preview",
          messages=[
              {"role": "system", "content": "You are a world-class surrealist writer. Your job is to take your student's rough initial draft of the first chapter of their surrealist novel, and rewrite it to be significantly better, with much more detail."},
              {"role": "user", "content": f"Here is the high-level plot you asked your student to follow: {plot}\n\nHere is the first chapter they wrote: {response.choices[0].message.content}\n\nNow, rewrite the first chapter of this novel, in a way that is far superior to your student's chapter. It should still follow the exact same plot, but it should be far more detailed, much longer, and more engaging. Here is a description of the writing style you should use: `{writing_style}`"}
          ]
      )

    print_step_costs(response, "gpt-4-1106-preview")

    return improved_response.choices[0].message.content


def write_chapter(previous_chapters, plot, chapter_title):
        try:
            i = random.randint(1,2242)
            response = openai.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=[
                    {"role": "system", "content": "You are a world-class surrealist writer."},
                    {"role": "user", "content": f"Plot: {plot}, Previous Chapters: {previous_chapters}\n\n--\n\nWrite the next chapter of this novel, following the plot and taking in the previous chapters as context. Here is the plan for this chapter: {chapter_title}\n\nWrite it beautifully. Include only the chapter text. There is no need to rewrite the chapter name."}
                ]
            )

            print_step_costs(response, "gpt-4-1106-preview")

            return response.choices[0].message.content

        except:
            response = openai.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=[
                    {"role": "system", "content": "You are a world-class surrealist writer."},
                    {"role": "user", "content": f"Plot: {plot}, Previous Chapters: {previous_chapters}\n\n--\n\nWrite the next chapter of this novel, following the plot and taking in the previous chapters as context. Here is the plan for this chapter: {chapter_title}\n\nWrite it beautifully. Include only the chapter text. There is no need to rewrite the chapter name."}
                ]
            )

            print_step_costs(response, "gpt-4-1106-preview")

            return response.choices[0].message.content



def generate_storyline(prompt, num_chapters):
    print("Generating storyline with chapters and high-level details...")
    json_format = """[{"Chapter CHAPTER_NUMBER_HERE - CHAPTER_TITLE_GOES_HERE": "CHAPTER_OVERVIEW_AND_DETAILS_GOES_HERE"}, ...]"""
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a world-class surrealist writer. Your job is to write a detailed storyline, complete with chapters, for a fantasy novel. Don't be flowery -- you want to get the message across in as few words as possible. But those words should contain lots of information."},
            {"role": "user", "content": f'Write a fantastic storyline with {num_chapters} chapters and high-level details based on this plot: {prompt}.\n\nDo it in this list of dictionaries format {json_format}'}
        ]
    )

    print_step_costs(response, "gpt-4-1106-preview")

    improved_response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a world-class surrealist writer. Your job is to take your student's rough initial draft of the storyline of a fantasy novel, and rewrite it to be significantly better."},
            {"role": "user", "content": f"Here is the draft storyline they wrote: {response.choices[0].message.content}\n\nNow, rewrite the storyline, in a way that is far superior to your student's version. It should have the same number of chapters, but it should be much improved in as many ways as possible. Remember to do it in this list of dictionaries format {json_format}"}
        ]
    )

    print_step_costs(improved_response, "gpt-4-1106-preview")

    return improved_response.choices[0].message.content

def write_to_file(prompt, content):

    # Create a directory for the prompts if it doesn't exist
    if not os.path.exists('prompts'):
        os.mkdir('prompts')

    # Replace invalid characters for filenames
    valid_filename = ''.join(c for c in prompt if c.isalnum() or c in (' ', '.', '_')).rstrip()
    file_path = f'prompts/{valid_filename}.txt'

    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f'Output for prompt "{prompt}" has been written to {file_path}\n')


def write_novel(prompt, num_chapters, writing_style):
    plots = generate_plots(prompt)
    print('generated plots')

    best_plot = select_most_engaging(plots)
    print('selected best plot')

    improved_plot = improve_plot(best_plot)
    print('plot improved')

    title = get_title(improved_plot)
    print('title generated')

    storyline = generate_storyline(improved_plot, num_chapters)
    print('storyline generated')
    chapter_titles = ast.literal_eval(storyline)


    novel = f"Storyline:\n{storyline}\n\n"

    first_chapter = write_first_chapter(storyline, chapter_titles[0], writing_style.strip())
    print('first chapter written')
    novel += f"Chapter 1:\n{first_chapter}\n"
    chapters = [first_chapter]

    for i in range(num_chapters - 1):
        print(f"Writing chapter {i+2}...")  # + 2 because the first chapter was already added

        chapter = write_chapter(novel, storyline, chapter_titles[i+1])
        try:
            if len(str(chapter)) < 100:
                print('Length minimum not hit. Trying again.')
                chapter = write_chapter(novel, storyline, chapter_titles[i+1])
        except:
            chapter = write_chapter(novel, storyline, chapter_titles[i+1])

        novel += f"Chapter {i+2}:\n{chapter}\n"
        chapters.append(chapter)

    with open('output.txt', 'w') as file:
        # Write data to the file
        file.write(f"Title: {title}\n")
        file.write("Chapters:\n")
        for chapter_title, chapter_content in zip(chapter_titles, chapters):
            file.write(f"Chapter: {chapter_title}\n")
            file.write(chapter_content)
            file.write('\n')

    return novel, title, chapters, chapter_titles


Epub Generation Functions Setup: Formatting, and Generation

In [27]:
# @title
def create_epub(title, author, chapters, cover_image_path='cover.png'):
    book = epub.EpubBook()

    # Set metadata
    book.set_identifier('id8797656')
    book.set_title(title)
    book.set_language('en')
    book.add_author(author)

    # Add cover image
    with open(cover_image_path, 'rb') as cover_file:
        cover_image = cover_file.read()
    book.set_cover('cover.png', cover_image)

    # Create chapters and add them to the book
    epub_chapters = []
    for i, chapter_dict in enumerate(chapters):
        full_chapter_title = list(chapter_dict.keys())[0]
        chapter_content = list(chapter_dict.values())[0]
        if ' - ' in full_chapter_title:
            chapter_title = full_chapter_title.split(' - ')[1]
        else:
            chapter_title = full_chapter_title

        chapter_file_name = f'chapter_{i+1}.xhtml'
        epub_chapter = epub.EpubHtml(title=chapter_title, file_name=chapter_file_name, lang='en')

        # Add paragraph breaks
        formatted_content = ''.join(f'<p>{paragraph.strip()}</p>' for paragraph in chapter_content.split('\n') if paragraph.strip())

        epub_chapter.content = f'<h1>{chapter_title}</h1>{formatted_content}'
        book.add_item(epub_chapter)
        epub_chapters.append(epub_chapter)


    # Define Table of Contents
    book.toc = (epub_chapters)

    # Add default NCX and Nav files
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # Define CSS style
    style = '''
    @namespace epub "http://www.idpf.org/2007/ops";
    body {
        font-family: Cambria, Liberation Serif, serif;
    }
    h1 {
        text-align: left;
        text-transform: uppercase;
        font-weight: 200;
    }
    '''

    # Add CSS file
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)

    # Create spine
    book.spine = ['nav'] + epub_chapters

    # Save the EPUB file
    epub.write_epub(f'{title}.epub', book)

Cover Generation Functions Setup

In [28]:
# @title
def generate_cover_prompt(plot):
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a creative assistant that writes a spec for the cover art of a book, based on the book's plot."},
            {"role": "user", "content": f"Plot: {plot}\n\n--\n\nDescribe the cover we should create, based on the plot. This should be two sentences long, maximum."}
        ]
    )
    return response.choices[0].message.content


def create_cover_image(plot):

  plot = str(generate_cover_prompt(plot))

# Retrieve the Stability AI API key
  STABILITY_API_KEY = userdata.get('STABILITY_API_KEY')

  engine_id = "stable-diffusion-xl-beta-v2-2-2"
  api_host = os.getenv('API_HOST', 'https://api.stability.ai')
  api_key = STABILITY_API_KEY  # Use the retrieved secret key

  if api_key is None:
      raise Exception("Missing Stability API key.")

  response = requests.post(
      f"{api_host}/v1/generation/{engine_id}/text-to-image",
      headers={
          "Content-Type": "application/json",
          "Accept": "application/json",
          "Authorization": f"Bearer {api_key}"
      },
      json={
          "text_prompts": [
              {
                  "text": plot
              }
          ],
          "cfg_scale": 7,
          "clip_guidance_preset": "FAST_BLUE",
          "height": 768,
          "width": 512,
          "samples": 1,
          "steps": 30,
      },
  )

  if response.status_code != 200:
      raise Exception("Non-200 response: " + str(response.text))

  data = response.json()

  for i, image in enumerate(data["artifacts"]):
      with open(f"/content/cover.png", "wb") as f: # replace this if running locally, to where you store the cover file
          f.write(base64.b64decode(image["base64"]))


# **Generate the Novel!**

Generate Novel Content

In [21]:
# @title
novel, title, chapters, chapter_titles = write_novel(prompt, num_chapters, writing_style)

# Replace chapter descriptions with body text in chapter_titles
for i, chapter in enumerate(chapters):
    chapter_number_and_title = list(chapter_titles[i].keys())[0]
    chapter_titles[i] = {chapter_number_and_title: chapter}

Step Cost (OpenAI): 0.024669999999999997
generated plots
Step Cost (OpenAI): 0.030889999999999997
selected best plot
Step Cost (OpenAI): 0.02787
plot improved
Step Cost (OpenAI): 0.007450000000000001
title generated
Generating storyline with chapters and high-level details...
Step Cost (OpenAI): 0.026569999999999996
Step Cost (OpenAI): 0.03325
storyline generated
Step Cost (OpenAI): 0.02874
Step Cost (OpenAI): 0.02874
first chapter written
Writing chapter 2...
Step Cost (OpenAI): 0.04974
Writing chapter 3...
Step Cost (OpenAI): 0.06094000000000001
Writing chapter 4...
Step Cost (OpenAI): 0.0717
Writing chapter 5...
Step Cost (OpenAI): 0.07961
Writing chapter 6...
Step Cost (OpenAI): 0.0897
Writing chapter 7...
Step Cost (OpenAI): 0.09735
Writing chapter 8...
Step Cost (OpenAI): 0.10604000000000001
Writing chapter 9...
Step Cost (OpenAI): 0.11522
Writing chapter 10...
Step Cost (OpenAI): 0.12349


Generate Cover and Create ePub

In [22]:
# @title
create_cover_image(str(chapter_titles))

# Create the EPUB file
create_epub(title, 'AuthorGPT', chapter_titles, '/content/cover.png')

### Copy to your Google Drive

In [30]:
# @title
import os
import shutil
from datetime import datetime
from google.colab import files


# Function to download files from a directory
def download_files_from_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            files.download(file_path)

# Directories
source_directory = '/content/'
drive_mount_point = '/content/drive/'
base_destination_directory = os.path.join(drive_mount_point, 'MyDrive/AI/AuthorGPT/')

# Check if the destination directory exists on Google Drive
if not os.path.exists(base_destination_directory):
    base_destination_directory = '/content/'

# Make assignment statement
ebook_title = title

# Create a timestamped directory with the eBook title
current_date = datetime.now().strftime("%Y_%m_%d")
destination_directory = os.path.join(base_destination_directory, f"{ebook_title}_{current_date}")
os.makedirs(destination_directory, exist_ok=True)

# Copy all non-directory files
for filename in os.listdir(source_directory):
    source_file = os.path.join(source_directory, filename)
    destination_file = os.path.join(destination_directory, filename)
    if os.path.isfile(source_file):
        shutil.copy2(source_file, destination_file)

# Confirmation messages
print(f'All non-directory files from {source_directory} copied to {destination_directory}')
print('Click below to download the eBook.')

# Generate download link(s) for the files
download_files_from_directory(destination_directory)


All non-directory files from /content/ copied to /content/drive/MyDrive/AI/AuthorGPT/Nasal Nexus: The Last Booger_2023_11_21
Click below to download the eBook.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>